<a href="https://colab.research.google.com/github/lwerth/SI/blob/master/replacement_loop.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
# Laura Werthmann
# here I try to implement a find and replace for loop to reduce the classification tags 
# hopefully this helps with classification accuracy

In [35]:
#code taken from https://www.kaggle.com/hsankesara/news-classification-using-han/notebook
#and
#https://colab.research.google.com/drive/1Q2tf4wSQRVq_opjWAYzpbYpPpzcnDaje?usp=sharing
#and
#https://colab.research.google.com/drive/1Q2tf4wSQRVq_opjWAYzpbYpPpzcnDaje?usp=sharing

##Set-up

In [36]:
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer,  text_to_word_sequence
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback, ModelCheckpoint
from keras.utils.np_utils import to_categorical
from keras.layers import Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed, Dropout
from keras import backend as K
from keras import optimizers
from keras.models import Model
import nltk
import re
import matplotlib.pyplot as plt
import sys
from sklearn.metrics import roc_auc_score
from nltk import tokenize
import seaborn as sns
import json
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

In [37]:
from sklearn.utils import shuffle

In [38]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [39]:
%cd /content/drive
!ls


/content/drive
'My Drive'


In [40]:
import pandas as pd

##Pre-Processing

Let's read in the csv file created from the spider crawl. Make sure encoding is utf-8 and we are dropping rows with no observations. 

In [41]:
#df = pd.read_csv('My Drive/Projects/InSightCrime/Insight_Crime/scrape/insight/InsightCrime_News.csv' ).dropna()
df = pd.read_csv('My Drive/Projects/InSightCrime/Insight_Crime/scrape/insight/InsightCrime_News.csv', encoding='utf-8').dropna()
new_tags = pd.read_excel('My Drive/Projects/InSightCrime/Insight_Crime/categories_v5.xlsx', encoding='utf-8')



Review the number of "unique" tags per article. some tags could still be repeated because of their list-like nature.

In [42]:
len(df.tags.unique())

5365

In [43]:
df.head()

,author,content,date,tags,title,type,url
0,Max Radwin,Protests broke out in Bolivia last year in the...,2020-03-09,"Bolivia,Cocaine,Elites and Organized Crime,Inf...",Bolivia Investigations Target Members of Evo M...,Analysis,https://www.insightcrime.org/news/analysis/bol...
1,Chris Dalby,Increased security of containers entering the ...,2020-02-27,"Brazil,Cocaine,European Organized Crime,Securi...",Can Port of Antwerp Truly Stem Cocaine Enterin...,Analysis,https://www.insightcrime.org/news/analysis/por...
2,Chris Dalby and Charlotte Dierkes,The sight of children bearing arms in Guerrero...,2020-02-17,"Homicides,Mexico",Los Ardillos Continue Terrorizing Indigenous C...,Analysis,https://www.insightcrime.org/news/analysis/los...
3,Maria Fernanda Ramírez,Helena is one of the many former FARC members ...,2020-02-04,"Colombia,FARC,FARC peace,Gender and Organized ...",Former FARC Guerrillas Recognized as Victims o...,Brief,https://www.insightcrime.org/news/brief/colomb...
4,Maria Alejandra Navarrete,"Luis Del Río Jiménez, alias “el Tío,” or “Seño...",2020-01-21,"Colombia,Elites and Organized Crime",Was ‘Señor T’ One of Colombia’s Invisible Drug...,Analysis,https://www.insightcrime.org/news/analysis/sen...


In [44]:
new_tags.head()

,actor,actor detail,action,action definition,country,Tags
0,Águilas Negras,NaN,Arms Trafficking,"The purchase, sale, or illegal transportation ...",Argentina,Arms Trafficking
1,Amigos dos Amigos,NaN,Contraband,Refers to items that are illegaly sold or tran...,Belize,Contraband
2,AUC,NaN,Corruption,This category corresponds to the Insight Crime...,Bolivia,Corruption
3,Autodefensas,NaN,Counterfeit,"Fake items, includes: fake money, consumer goo...",Brazil,Counterfeit
4,Bagdad,NaN,Criminal Migration,"The relocation of armed groups, gangs, or cart...",Canada,Criminal Migration


In [45]:
df['text'] = df['title'] +'. ' +df['content']
df.head()

,author,content,date,tags,title,type,url,text
0,Max Radwin,Protests broke out in Bolivia last year in the...,2020-03-09,"Bolivia,Cocaine,Elites and Organized Crime,Inf...",Bolivia Investigations Target Members of Evo M...,Analysis,https://www.insightcrime.org/news/analysis/bol...,Bolivia Investigations Target Members of Evo M...
1,Chris Dalby,Increased security of containers entering the ...,2020-02-27,"Brazil,Cocaine,European Organized Crime,Securi...",Can Port of Antwerp Truly Stem Cocaine Enterin...,Analysis,https://www.insightcrime.org/news/analysis/por...,Can Port of Antwerp Truly Stem Cocaine Enterin...
2,Chris Dalby and Charlotte Dierkes,The sight of children bearing arms in Guerrero...,2020-02-17,"Homicides,Mexico",Los Ardillos Continue Terrorizing Indigenous C...,Analysis,https://www.insightcrime.org/news/analysis/los...,Los Ardillos Continue Terrorizing Indigenous C...
3,Maria Fernanda Ramírez,Helena is one of the many former FARC members ...,2020-02-04,"Colombia,FARC,FARC peace,Gender and Organized ...",Former FARC Guerrillas Recognized as Victims o...,Brief,https://www.insightcrime.org/news/brief/colomb...,Former FARC Guerrillas Recognized as Victims o...
4,Maria Alejandra Navarrete,"Luis Del Río Jiménez, alias “el Tío,” or “Seño...",2020-01-21,"Colombia,Elites and Organized Crime",Was ‘Señor T’ One of Colombia’s Invisible Drug...,Analysis,https://www.insightcrime.org/news/analysis/sen...,Was ‘Señor T’ One of Colombia’s Invisible Drug...


Check for reduncancies and replace them.

The accents are not counted as redundant here

In [46]:
df.tags.unique()

array(['Bolivia,Cocaine,Elites and Organized Crime,Infographics',
       'Brazil,Cocaine,European Organized Crime,Security Policy,Suriname',
       'Homicides,Mexico', ...,
       'Colombia,Oficina de Envigado,Sebastián (oficina)',
       'Cartel de Juárez,Cartel de Sinaloa,El Chapo,Homicidios,México',
       'Costa Rica,Lavado de dinero,Nicaragua'], dtype=object)

In [65]:
len(df.tags.unique())

5361

In [64]:
#df.tags = df.tags.map(lambda x: "Juarez" if x == "Juárez" else x)
df.tags = df.tags.map(lambda x: "Elites and Organized Crime" if x == "Élites y Crimen Organizado" else x)
df.tags = df.tags.map(lambda x: "Mexico" if x == "México" else x)
df.tags = df.tags.map(lambda x: "Brazil" if x == "Brasil" else x)
#df.tags = df.tags.map(lambda x: "Human Rights" if x == "Derechos Humanos" else x)
df.tags = df.tags.map(lambda x: "Peru" if x == "Perú" else x)
#df.tags = df.tags.map(lambda x: "Sinaloa Cartel" if x == "Cartel de Sinaloa" else x)
# df.tags = df.tags.map(lambda x: "FARC peace" if x == "Paz FARC" else x)
# df.tags = df.tags.map(lambda x: "Money Laudering" if x == "Lavado de dinero" else x)

In [54]:
df.tags.unique()

array(['Bolivia,Cocaine,Elites and Organized Crime,Infographics',
       'Brazil,Cocaine,European Organized Crime,Security Policy,Suriname',
       'Homicides,Mexico', ...,
       'Colombia,Oficina de Envigado,Sebastián (oficina)',
       'Cartel de Juárez,Cartel de Sinaloa,El Chapo,Homicidios,México',
       'Costa Rica,Lavado de dinero,Nicaragua'], dtype=object)

In [52]:
len(df.tags.unique())

5363

In [58]:
df['cats'] = df.tags.str.split(",", expand=False)

In [72]:
df['cats'][0]


['Bolivia', 'Cocaine', 'Elites and Organized Crime', 'Infographics']

In [31]:
# df.cats = df.cats.map(lambda x: "Elites and Organized Crime" if x == "Élites y Crimen Organizado" else x)
# df.cats = df.cats.map(lambda x: "Mexico" if x == "México" else x)
# df.cats = df.cats.map(lambda x: "Brazil" if x == "Brasil" else x)
# df.cats = df.cats.map(lambda x: "Human Rights" if x == "Derechos Humanos" else x)
# df.cats = df.cats.map(lambda x: "Peru" if x == "Perú" else x)
# df.cats = df.cats.map(lambda x: "Sinaloa Cartel" if x == "Cartel de Sinaloa" else x)
# df.cats = df.cats.map(lambda x: "FARC peace" if x == "Paz FARC" else x)

In [75]:
def cambios(nuevo, original):
  df['cats'].map(lambda x: str(nuevo) if x == str(original) else x)
  return None

  cambios("Elites and Organized Crime","Élites y Crimen Organizado")
  cambios('corruption', 'Elites and Organized Crime')
  cambios("Mexico", "México")
  cambios("Brazil","Brasil")
  cambios("Human Rights","Derechos Humanos")
  cambios("Peru","Perú")
  cambios("Sinaloa Cartel","Cartel de Sinaloa")
  cambios("FARC peace","Paz FARC")
  cambios("Money Laundering", "Lavado de dinero")


In [84]:
categories = df['cats']
categorias = df.tags

def cambios(nuevo, original):
  for i in (categorias.index):
    categorias.map(lambda x: str(nuevo) if x == str(original) else x)
    print('we changed', original, "to", nuevo)
    return categorias.map(lambda x: str(nuevo) if x == str(original) else x)

  cambios("Elites and Organized Crime","Élites y Crimen Organizado")
  cambios('corruption', 'Elites and Organized Crime')
  cambios("Mexico", "México")
  cambios("Brazil","Brasil")
  cambios("Human Rights","Derechos Humanos")
  cambios("Peru","Perú")
  cambios("Sinaloa Cartel","Cartel de Sinaloa")
  cambios("FARC peace","Paz FARC")
  cambios("Money Laundering", "Lavado de dinero")

In [85]:
df.tags.unique

<bound method Series.unique of 0        Bolivia,Cocaine,Elites and Organized Crime,Inf...
1        Brazil,Cocaine,European Organized Crime,Securi...
2                                         Homicides,Mexico
3        Colombia,FARC,FARC peace,Gender and Organized ...
4                      Colombia,Elites and Organized Crime
                               ...                        
15764                            El Loco Barrera,Venezuela
15765                  Frontera EU/México,Tráfico de armas
15766                Costa Rica,Lavado de dinero,Nicaragua
15767                                               Mexico
15768                                              Uruguay
Name: tags, Length: 15655, dtype: object>

In [ ]:
new_tags = new_tags[['actor','actor detail', 'action', 'country','Tags']]
new_tags['combined_tags'] = new_tags['Tags']
new_tags = new_tags[['actor', 'actor detail', 'action', 'country', 'combined_tags']]
new_tags.head()

,actor,actor detail,action,country,combined_tags
0,Águilas Negras,NaN,Arms Trafficking,Argentina,Arms Trafficking
1,Amigos dos Amigos,NaN,Contraband,Belize,Contraband
2,AUC,NaN,Corruption,Bolivia,Corruption
3,Autodefensas,NaN,Counterfeit,Brazil,Counterfeit
4,Bagdad,NaN,Criminal Migration,Canada,Criminal Migration


In [ ]:
#merge the datasets
new = pd.concat([df, new_tags], axis=1, sort=False)
new.head()

,author,content,date,tags,title,type,url,text,cats,actor,action,country,combined_tags
0,Max Radwin,Protests broke out in Bolivia last year in the...,2020-03-09,"Bolivia,Cocaine,Elites and Organized Crime,Inf...",Bolivia Investigations Target Members of Evo M...,Analysis,https://www.insightcrime.org/news/analysis/bol...,Bolivia Investigations Target Members of Evo M...,"[Bolivia, Cocaine, Elites and Organized Crime,...",Águilas Negras,Arms Trafficking,Argentina,Arms Trafficking
1,Chris Dalby,Increased security of containers entering the ...,2020-02-27,"Brazil,Cocaine,European Organized Crime,Securi...",Can Port of Antwerp Truly Stem Cocaine Enterin...,Analysis,https://www.insightcrime.org/news/analysis/por...,Can Port of Antwerp Truly Stem Cocaine Enterin...,"[Brazil, Cocaine, European Organized Crime, Se...",Amigos dos Amigos,Contraband,Belize,Contraband
2,Chris Dalby and Charlotte Dierkes,The sight of children bearing arms in Guerrero...,2020-02-17,"Homicides,Mexico",Los Ardillos Continue Terrorizing Indigenous C...,Analysis,https://www.insightcrime.org/news/analysis/los...,Los Ardillos Continue Terrorizing Indigenous C...,"[Homicides, Mexico]",AUC,Corruption,Bolivia,Corruption
3,Maria Fernanda Ramírez,Helena is one of the many former FARC members ...,2020-02-04,"Colombia,FARC,FARC peace,Gender and Organized ...",Former FARC Guerrillas Recognized as Victims o...,Brief,https://www.insightcrime.org/news/brief/colomb...,Former FARC Guerrillas Recognized as Victims o...,"[Colombia, FARC, FARC peace, Gender and Organi...",Autodefensas,Counterfeit,Brazil,Counterfeit
4,Maria Alejandra Navarrete,"Luis Del Río Jiménez, alias “el Tío,” or “Seño...",2020-01-21,"Colombia,Elites and Organized Crime",Was ‘Señor T’ One of Colombia’s Invisible Drug...,Analysis,https://www.insightcrime.org/news/analysis/sen...,Was ‘Señor T’ One of Colombia’s Invisible Drug...,"[Colombia, Elites and Organized Crime]",Bagdad,Criminal Migration,Canada,Criminal Migration


##Replacement Tries


In [ ]:
#Sample Code

for i in range(len(a)):
    # you can do intersection/comparisons more easily with sets
    set_a = set(a[i])
    set_b = set(b[i])
    this_c_diff = set_a.difference(set_b)
    c_diff.append(list(this_c_diff))
print(np.array(c_diff))

#///////////////////////////////////////////////////
for i in range(len(series.index)):
    for key in label_dict:
        if series.iloc[i] in label_dict[key]:
            series_copy.iloc[i] = key

In [ ]:
'''
I want to reduce the article tags to already decided genre tags. 
Each cell can have multiple tags, I want to single out each tag and replace it 
with a reduced tag if necessary.
They don't match up evenly so I need to find a way to replace the tags. 
They are originally separated into genres, but I combined them to keep 
one list of all the redecued tags.
I might go one genre at a time, i.e. actor, action, country
I'm going to start with country because that might be the easiest to work with

'''

def reduce_replace(series, country):
  country = new_tags['country']
  series = df['cats']
  series_copy = []
  #tag_country = []

  
  for i in range(len(series.index)):
    for key in country:
      if series.iloc[i] in country[key]:
        series_copy.iloc[i] = key
      print(np.array(reduce_replace))
      print(series_copy)


In [ ]:
# new_tags['tags'] = new_tags.combined_tags
# df_outer = pd.merge(df, new_tags, on= 'tags', how='outer', suffixes=('_og', '_new') )
# df_outer

In [ ]:
string = df.cats
actor_mappings = new_tags.combined_tags.iloc

def keymap_replace(
    string: str, 
    actor_mappings: dict,
    *,
    lower_keys=False,
    lower_values=False,
    lower_string=False,
) -> str:
    replaced_string = string.lower() if lower_string else string
    for character, replacement in actor_mappings.items():
        replaced_string = replaced_string.replace(
            character.lower() if lower_keys else character,
            replacement.lower() if lower_values else replacement
        )
    return replaced_string

print(keymap_replace(string, {
    'El Chayo' : 'Caballeros Templarios',
     'La Tuta': 'Caballeros Templarios',
    'El Mencho': 'Cartel de Jalisco',
    'El Ingeniero (Juarez)': 'Cartel de Juárez',
    'El Viceroy' : 'Cartel de Juárez',
    'Walid Makled' : 'Cartel de los Soles',
    'El Turco' : 'Cartel de los Soles',
    'El Arabe' : 'Cartel de los Soles',
    'El Loco Barrera' : 'Rastrojos'
}))

0        [Bolivia, Cocaine, Elites and Organized Crime,...
1        [Brazil, Cocaine, European Organized Crime, Se...
2                                      [Homicides, Mexico]
3        [Colombia, FARC, FARC peace, Gender and Organi...
4                   [Colombia, Elites and Organized Crime]
                               ...                        
15764                         [El Loco Barrera, Venezuela]
15765               [Frontera EU/México, Tráfico de armas]
15766            [Costa Rica, Lavado de dinero, Nicaragua]
15767                                             [Mexico]
15768                                            [Uruguay]
Name: cats, Length: 15655, dtype: object


Look here for potential help:

http://pythoninthewyld.com/2018/03/12/dict-based-find-and-replace-deluxe/

https://stackoverflow.com/questions/19961432/find-and-replace-strings-in-a-python-dictionary

https://docs.python.org/3/library/stdtypes.html#str.translate

https://codereview.stackexchange.com/questions/97318/string-replacement-using-dictionaries

In [ ]:
#Frist attempt, need to figure out how to change the key, and not sure this acctually works
class tag_replacer():
    def __init__(self, text, reduced):
        self.text = original
        self.iterate(reduced)

    def replace_kv(self, _dict):
        """Replace any occurrence of a value with the key"""

        for key, value in _dict.items():
            label = """[[ {0} ]]""".format(key) #new_tags['combined_tags'].key(value) 
            self.text = self.text.replace(value, label)
            return self.text

    def iterate(self, reduced):
        """Iterate over each dict object in a given list of dicts, `reduced` """

        for _dict in reduced:
            self.text = self.replace_kv(_dict)
        return self.text

original = df.cats
reduced = [{'action':new_tags['action']}, {'actor': new_tags['actor']}, {'country': new_tags['country']}]
processed = tag_replacer(original, reduced)
processed.text

0        [Bolivia, Cocaine, Elites and Organized Crime,...
1        [Brazil, Cocaine, European Organized Crime, Se...
2                                      [Homicides, Mexico]
3        [Colombia, FARC, FARC peace, Gender and Organi...
4                   [Colombia, Elites and Organized Crime]
                               ...                        
15764                         [El Loco Barrera, Venezuela]
15765               [Frontera EU/México, Tráfico de armas]
15766            [Costa Rica, Lavado de dinero, Nicaragua]
15767                                             [Mexico]
15768                                            [Uruguay]
Name: cats, Length: 15655, dtype: object

In [ ]:
#second attempt, needs a little help but this might be a good start. 
tags_sincasa = []

actor_list = new_tags['actor']
action_list = new_tags['action']
country_list = new_tags['country']

#start with intersections and comparisons
for line in new_tags['actor']:
  set_actor = set(actor_list[line])
  set_tags = set(df.tags[line])

  non_actor = set_actor.difference(set_tags)
  tags_sincasa.append(list(non_actor))

for line in new_tags['action']:
  set_action = set(action_list[line])
  set_tags = set(df.tags[line])
  non_action = set_action.difference(set_tags)
  tags_sincasa.append(list(non_action))

for line in new_tags['country']:
  set_country = set(country_list[line])
  set_tags = set(df.tags[line])
  non_country = set_country.difference(set_tags)
  tags_sincasa.append(list(non_country))

print(np.array(tags_reduced))

In [ ]:
new_tags.head()

,actor,action,country,Tags
0,Águilas Negras,Arms Trafficking,Argentina,Arms Trafficking
1,Amigos dos Amigos,Contraband,Belize,Contraband
2,AUC,Corruption,Bolivia,Corruption
3,Autodefensas,Counterfeit,Brazil,Counterfeit
4,Bagdad,Criminal Migration,Canada,Criminal Migration


What I'm trying to do is create a dictionary with the reduced categories that will replace the og categories as keys (key labels), and mapped to those keys will be the lists containing all the og categories we want to replace with that particular key label. 

Then, we can loop over the labels and have an inner loop traversing over the dictionary checking to see if the og category is in the list for each key. If it is, replace the og category in the labels with the key label... but I keep getting stuck.

In [ ]:
series = df['cats']
label_dict = new_tags['Tags']
series_copy = []

for i in range(len(series.index)):
    for key in label_dict:
        if series.iloc[i] in label_dict[key]:
            series_copy.iloc[i] = key

KeyError: ignored

In [ ]:
new_tags_transp = new_tags.T

In [ ]:
new_tags_transp.head

<bound method NDFrame.head of                       0                  1    ...  104        105
actor      Águilas Negras  Amigos dos Amigos  ...  NaN        NaN
action   Arms Trafficking         Contraband  ...  NaN        NaN
country         Argentina             Belize  ...  NaN        NaN
Tags     Arms Trafficking         Contraband  ...  USA  Venezuela

[4 rows x 106 columns]>

I'm trying to implement the for loop in two different way with these next few scripts, I'm just not sure how to continue or fix them ... Doing another google search, I think I might have the correct language now.

In [ ]:
def reduce_tags(self, action_tags, actor_tags, country_tags, reduced_column):
  tags_sincasa = []
  tags_actor = []
  tags_action = []
  tags_country = []
  reduced_column = []
  new_tags_file = csv.writer(new_tags, delimiter = ';', lineterminator='\n')

  og_tags = df['tags']
  
  for line in og_tags:
    tags_action[line].replace(action_tags[line])
    tags_actor[line].replace(actor_tags[line])
    tags_country[line].replace(country_tags[line])

  new_tags_file.writerows(reduced_column)

In [ ]:
#def replce(self, )
tags_sincasa = []
tags_actor = []
tags_action = []
tags_country = []

actor_list = new_tags['actor'].tolist()
action_list = new_tags['action'].tolist()
country_list = new_tags['country'].tolist()

# with open(new_tags, 'r') as infile:
#   

for line in new_tags:
  # tags_actor[line].append(new_tags.actor[line])
  # tags_action[line].append(new_tags.action[line])
  # tags_country[line].append(new_tags.country[line])


  set_action = set(actor_list[line])
  set_actor = set(action_list[line])
  set_country = set(country_list[line])
  set_tags = set(df.tags[line])

  non_action = set_action.difference(set_tags)
  tags_sincasa.append(list(non_action))

print(np.array(tags_reduced))


def fill_csv(self, array_urls, array_dates, csv_file_path):
    result_array = []
    array_length = str(len(array_dates))

    # We fill the CSV file
    file = open(csv_file_path, "w")
    csv_file = csv.writer(file, delimiter=';', lineterminator='\n')

    # We merge the two arrays in one

    for i in array_length:
        result_array[i][0].append(array_urls[i])
        result_array[i][1].append(array_dates[i])
        i += 1

    csv_file.writerows(result_array)



# for i in range(len(df.tags)):
#   if (df.tags[i] == new_tags.action[i]):
#     tags_reduced = new_tags.action
#   elif (df.tags == new_tags.actor):
#     tags_reduced = new_tags.actor
#   elif (df.tags == new_tags.country):
#     tags_reduced = new_tags.country
# print(tags_reduced.head())

TypeError: ignored

In [ ]:
# for i in lines:
#     temp = str(i)
#     temp2 = temp.replace(' ', ', ',2)
#     linesNew.append(temp2)

#lines = df.readlines()
linesNew = []
temp = None


for lines in df.tags:
  temp = str(df.cats)
  temp2 = lines.replace(df.cats, new_tags.action)
  linesNew.append(temp2)

In [ ]:
from pandas import DataFrame

# names = {'First_name': ['Jon','Bill','Maria','Emma']}
# df = DataFrame(names,columns=['First_name'])

# df.loc[(df['First_name'] == 'Bill') | (df['First_name'] == 'Emma'), 'name_match'] = 'Match'  
# df.loc[(df['First_name'] != 'Bill') & (df['First_name'] != 'Emma'), 'name_match'] = 'Mismatch'  

# print (df)

# new_tags.loc[(new_tags['actor'] == df.tags), 'Reduced'] = new_tags['actor']
# new_tags.head()

In [ ]:
df.tail()

,author,content,date,tags,title,type,url,text,cats
15764,Elyssa Pachico,SHARE\nLa captura de Daniel “El Loco” Barrera ...,2012-09-19,"El Loco Barrera,Venezuela",El arresto del ‘Loco Barrera’ muestra a Venezu...,Análisis,https://es.insightcrime.org/noticias/analisis/...,El arresto del ‘Loco Barrera’ muestra a Venezu...,"[El Loco Barrera, Venezuela]"
15765,Daniela Castro,SHARE\nEl informe\nelaborado por el inspector ...,2012-09-20,"Frontera EU/México,Tráfico de armas",Veredicto sobre la Operación ‘Rápido y Furioso’,Noticias del día,https://es.insightcrime.org/noticias/noticias-...,Veredicto sobre la Operación ‘Rápido y Furioso...,"[Frontera EU/México, Tráfico de armas]"
15766,Daniela Castro,SHARE\nFalsos periodistas de la cadena Televis...,2012-09-20,"Costa Rica,Lavado de dinero,Nicaragua",Falsos Periodistas Acusados por los Delitos de...,Noticias del día,https://es.insightcrime.org/noticias/noticias-...,Falsos Periodistas Acusados por los Delitos de...,"[Costa Rica, Lavado de dinero, Nicaragua]"
15767,Andres Ortiz,"SHARE\nEn lo corrido del año, la Secretaría de...",2012-09-21,Mexico,Aproximadamente 8.000 Detenidos por Narcotráfi...,Noticias del día,https://es.insightcrime.org/noticias/noticias-...,Aproximadamente 8.000 Detenidos por Narcotráfi...,[Mexico]
15768,Andres Ortiz,SHARE\nLuego de retirarse del país en 1994 por...,2012-09-21,Uruguay,La DEA Vuelve a Pisar Uruguay,Noticias del día,https://es.insightcrime.org/noticias/noticias-...,La DEA Vuelve a Pisar Uruguay. SHARE\nLuego de...,[Uruguay]


In [ ]:
df = df[['title', 'text', 'cats','tags','content']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15655 entries, 0 to 15768
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   title    15655 non-null  object
 1   text     15655 non-null  object
 2   cats     15655 non-null  object
 3   tags     15655 non-null  object
 4   content  15655 non-null  object
dtypes: object(5)
memory usage: 1.3+ MB


##Genres

I'm going to edit out the genres tab because it only has 253 rows when we need 15655. I though this would help with binarization but it does not.

In [ ]:
# select a column as series and then convert it into a column
list_of_names = df.cats.to_list()
#list_of_names

In [ ]:
all_genres = sum(list_of_names, [])
len(set(all_genres))

253

In [ ]:
all_genres = nltk.FreqDist(all_genres) 

# create dataframe
all_genres_df = pd.DataFrame({'Genre': list(all_genres.keys()), 
                              'Count': list(all_genres.values())})

all_genres_df.head

<bound method NDFrame.head of                           Genre  Count
0                       Bolivia    541
1                       Cocaine    141
2    Elites and Organized Crime    792
3                  Infographics    368
4                        Brazil    696
..                          ...    ...
248         Cartel los Perrones     12
249             Cartel de Texis     43
250             Víctor Carranza      8
251             Pijarbey Farfan      4
252         Sebastián (oficina)      1

[253 rows x 2 columns]>

Figure out how to reduce the 253 "genres", "tags", or categories and replace them with the ~60 chosen by the Insight Crime Group

In [ ]:
class TextLabeler():
    def __init__(self, text, lod):
        self.text = text
        self.iterate(lod)

    def replace_kv(self, _dict):
        """Replace any occurrence of a value with the key"""

        for key, value in _dict.iteritems():
            label = """[[ {0} ]]""".format(key)
            self.text = self.text.replace(value, label)
            return self.text

    def iterate(self, lod):
        """Iterate over each dict object in a given list of dicts, `lod` """

        for _dict in lod:
            self.text = self.replace_kv(_dict)
        return self.text

string = "Let's take a trip to Paris next January"
lod = [{'city':'Paris'}, {'month':'January'}]
processed = TextLabeler(string, lod)
processed.text
#Let's take a trip to [[ city ]] next [[ month ]]

AttributeError: ignored

In [ ]:
#Strings are immutable. replace returns a new string, which is what you have to insert into the linesNew list

for i in lines:
    temp = str(i)
    temp2 = temp.replace(' ', ', ',2)
    linesNew.append(temp2)

##LDA Try

Instead of using 

In [ ]:
!pip install gensim
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(400)

In [ ]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True